# SCOA

## Assignment 1
Union, Intersection, Complement and Difference of fuzzy sets

In [ ]:
import random
import string

In [ ]:
DEC = 2

In [ ]:
class FuzzySet:
  def __init__(self):
    self.set = dict()

  def add(self, key, value):
    self.set[key] = round(value, DEC)

  def get(self, key):
    if key in self.set.keys():
      return self.set[key]
    return 0

  def elements(self):
    return self.set.keys()

  def __str__(self):
    return str(self.set)
  
  def __repr__(self):
    return str(self.set)

  @classmethod
  def union(cls, A, B):
    C = FuzzySet()
    for elem in A.elements():
      C.add(elem, A.get(elem))
    for elem in B.elements():
      C.add(elem, max(B.get(elem), A.get(elem)))
    return C
  
  @classmethod
  def intersection(cls, A, B):
    C = FuzzySet()
    for elem in B.elements():
      if elem in A.elements():
        C.add(elem, min(B.get(elem), A.get(elem)))
    return C

  @classmethod
  def difference(cls, A, B):
    C = FuzzySet()
    for elem in A.elements():
      C.add(elem, min(A.get(elem), 1-B.get(elem)))
    return C
  
  @classmethod
  def complement(cls, A):
    X = FuzzySet()
    for elem in A.elements():
      X.add(elem, 1 - A.get(elem))
    return X

In [ ]:
class FuzzyRelation:
  def __init__(self, A = None, B = None):
    self.r_set = dict()
    if A is None or B is None:
      return
    for a in A.elements():
      self.r_set[a] = dict()
      for b in B.elements():
        self.r_set[a][b] = min(A.get(a), B.get(b))

  def row_elements(self):
    return self.r_set.keys()
  
  def col_elements(self):
    for k in self.row_elements():
      return self.r_set[k].keys()
    return []

  def get(self, r, c):
    if r not in self.row_elements():
      return 0
    if c not in self.col_elements():
      return 0
    return self.r_set[r][c]

  def add(self, r, c, val):
    if r not in self.row_elements():
      self.r_set[r] = dict()
    self.r_set[r][c] = round(val, DEC)
  
  def __str__(self):
    space = 5
    ret = "".center(space)
    for c in self.col_elements():
      ret += c.center(space)
    ret += "\n"
    for r in self.row_elements():
      ret += r.center(space)
      for c in self.col_elements():
        ret += str(self.get(r, c)).center(space)
      ret += "\n"
    return ret

  def __repr__(self):
    space = 5
    ret = "".center(space)
    for c in self.col_elements():
      ret += c.center(space)
    ret += "\n"
    for r in self.row_elements():
      ret += r.center(space)
      for c in self.col_elements():
        ret += str(self.get(r, c)).center(space)
      ret += "\n"
    return ret
  
  @classmethod
  def max_min_composition(cls, R1, R2):
    R3 = FuzzyRelation()
    for r in R1.row_elements():
      for c in R2.col_elements():
        val = 0
        for k in R1.col_elements():
          val = max(val, min(R1.get(r, k), R2.get(k, c)))
        R3.add(r, c, val)
    return R3

In [ ]:
def init_random(Universe):
  S = FuzzySet()
  for c in Universe:
    if random.choice([0, 1]):
      S.add(c, round(random.uniform(0.1, 1), DEC))
  return S

In [ ]:
S1 = init_random(string.ascii_uppercase)
S2 = init_random(string.ascii_uppercase)
S3 = init_random(string.ascii_uppercase)

In [ ]:
print("S1: ", S1)
print("S2: ", S2)
print("S3: ", S3)

In [ ]:
print("Union: ", FuzzySet.union(S1, S2))
print("Intersection: ", FuzzySet.intersection(S1, S2))
print("Difference: ", FuzzySet.difference(S1, S2))
print("Complement: ", FuzzySet.complement(S1))

In [ ]:
A = init_random(string.ascii_uppercase)
B = init_random(string.digits)
C = init_random(string.ascii_lowercase)

In [ ]:
print("A: ", A)
print("B: ", B)
print("C: ", C)

In [ ]:
R1 = FuzzyRelation(A, B)
R2 = FuzzyRelation(B, C)

In [ ]:
print("R1")
print(R1)
print("R2")
print(R2)

In [ ]:
print(FuzzyRelation.max_min_composition(R1, R2))

## Assignment 2:
Implement genetic algorithm for benchmark function

In [ ]:
import random

In [ ]:
def decimalToBinary(n):
    return "{0:b}".format(int(n)).rjust(CHROMOSOME_LENGTH,"0")
def binaryToDecimal(n):
    return int(n,2)

In [ ]:
def toss(p):
  return random.randint(0, 100) <= p

In [ ]:
def get_best_individual(individuals):
  best_individual = individuals[0]
  for i in individuals:
    if i.get_fitness() > best_individual.get_fitness():
      best_individual = i
  return best_individual

In [ ]:
class Individual:
  def __init__(self, chromosome):
    self.chromosome = chromosome
    self.value = binaryToDecimal(chromosome)
    self.length = len(chromosome)

  def get_fitness(self):
    return -(A - self.value)*(A - self.value)
  
  def mutate(self):
    new_chromosome = self.chromosome
    for i in range(self.length-2):
      if toss(MUTATION_PROBABILITY):
        bit = "0"
        if new_chromosome[i] == "0":
          bit = "1"
        new_chromosome = new_chromosome[:i] + bit + new_chromosome[i+1:]
    if toss(MUTATION_PROBABILITY):
      bit = "0"
      if new_chromosome[i] == "0":
        bit = "1"
      new_chromosome = new_chromosome[:-1] + bit
    self.chromosome = new_chromosome
    self.value = binaryToDecimal(self.chromosome)

  def __str__(self):
    ret = """
{
    "chromosome":""" + self.chromosome +"""
    "fitness":""" + str(self.get_fitness()) +"""
}"""
    return ret
  
  def __repr__(self):
    return self.chromosome

  @classmethod
  def gen_chromosome(cls, chromosome_length):
    chromosome = "".join([random.choice(["0", "1"]) for _ in range(chromosome_length)])
    return chromosome

  @classmethod
  def crossover(cls, individual1, individual2):
    k = random.randint(1, individual1.length - 2)
    new_chromosome1 = individual1.chromosome[:k] + individual2.chromosome[k:]
    new_chromosome2 = individual2.chromosome[:k] + individual1.chromosome[k:]
    return Individual(new_chromosome1), Individual(new_chromosome2)


In [ ]:
POPULATION_SIZE = 10

CHROMOSOME_LENGTH = 8

TOURNAMENT_SIZE = 2

CROSSOVER_PROBABILITY = 70
MUTATION_PROBABILITY = 10

NUM_GENERATIONS = 1000

In [ ]:
A = 100 # f(x) = (x-a)^2

In [ ]:
population = [Individual(Individual.gen_chromosome(CHROMOSOME_LENGTH)) for _ in range(POPULATION_SIZE)]
overall_best_individual = population[0]

In [ ]:
for _ in range(NUM_GENERATIONS):
  sampled_individuals = []
  for _ in range(0, POPULATION_SIZE):
    tournament_individuals = random.sample(population, TOURNAMENT_SIZE)
    best_individual = get_best_individual(tournament_individuals)
    sampled_individuals.append(best_individual)
  for i in range(0, POPULATION_SIZE, 2):
    if toss(CROSSOVER_PROBABILITY):
      individual1, individual2 = sampled_individuals[i], sampled_individuals[i+1]
      child1, child2 = Individual.crossover(individual1, individual2)
      sampled_individuals[i] = child1
      sampled_individuals[i+1] = child2
  population = sampled_individuals
  for i in range(POPULATION_SIZE):
    population[i].mutate()
  if get_best_individual(population).get_fitness() > overall_best_individual.get_fitness():
    overall_best_individual = get_best_individual(population)

In [ ]:
cur_best_individual = get_best_individual(population)
print("Current best individual:", cur_best_individual)
# print(population)
print("Overall best individual:", overall_best_individual)

## Assignment 3
Particle swarm optimization for benchmark function

In [ ]:
import numpy as np

In [ ]:
np.random.seed(42)

In [ ]:
class Particle:
  def __init__(self):
    self.fitness = round(np.random.rand(), 2)
    self.bfitness = round(np.random.rand(), 2)
    self.position = round(np.random.rand(), 2)
    self.bposition = round(np.random.rand(), 2)
    self.velocity = round(np.random.rand(), 2)

  def getFitness(self):
    return round(self.position*self.position, 2)

  def updateVelocity(self, overall_b_position):
    r1 = np.random.rand()
    r2 = np.random.rand()
    self.velocity = w*self.velocity + c1*r1*(self.bposition - self.position) + c2*r2*(overall_b_position - self.position)

  def updatePosition(self):
    self.position += self.velocity
    self.position = round(max(MIN, min(self.position, MAX)), 2)
  
  def update(self, overall_b_position):
    self.updateVelocity(overall_b_position)
    self.updatePosition()
    if self.getFitness() > self.bfitness:
      self.bposition = self.position
      self.bfitness = self.getFitness()
    return self.bfitness

  def getBPosition(self):
    return self.bposition
  
  def getBFitness(self):
    return self.bfitness

  def printString(self):
    ret = f"Fitness: {self.getFitness()}, bestFitness: {self.bfitness}"
    return ret;

In [ ]:
MIN = 0
MAX = 20

In [ ]:
w1 = 0.9
w2 = 0.4
c1 = 2
c2 = 2

In [ ]:
POPULATION_SIZE = 10
NUM_ITERS = 10

In [ ]:
w = (w2-w1)/(NUM_ITERS-1)

In [ ]:
swarm = [Particle() for _ in range(POPULATION_SIZE)]
best_particle = swarm[0]

In [ ]:
for iter in range(NUM_ITERS):
  print("Iteration ", iter)
  for i in range(len(swarm)):
    fitness = swarm[i].update(best_particle.getBPosition())
    print("\tParticle ", i, ": ", swarm[i].printString())
    if fitness > best_particle.getBFitness():
      best_particle = swarm[i]
  print("Best Particle: ", best_particle.printString())
  print()

## Assignment 4
Logic gates using Mc-Culoch-Pitts

In [ ]:
from abc import ABC, abstractmethod

In [ ]:
class Neuron(ABC):
  def __init__(self, weights):
    self.weights = weights

  def linearFunction(self, input_vector):
    val = self.weights[0]
    for i in range(2):
      val += input_vector[i] * self.weights[i+1]
    return val

  @abstractmethod
  def thresholdFunction(self, input_value):
    pass

  def process(self, input_vector):
    return self.thresholdFunction(self.linearFunction(input_vector))

In [ ]:
class ANDGate(Neuron):
  def __init__(self):
    super().__init__([-3, 2, 2])

  def thresholdFunction(self, value):
    return value > 0

In [ ]:
class ORGate(Neuron):
  def __init__(self):
    super().__init__([0, 1, 1])

  def thresholdFunction(self, value):
    return value >= 1

In [ ]:
class XORGate(Neuron):
  def __init__(self):
    super().__init__([0, -1, 1])

  def thresholdFunction(self, value):
    return value != 0

In [ ]:
class NORGate(Neuron):
  def __init__(self):
    super().__init__([0, 1, 1])

  def thresholdFunction(self, value):
    return value == 0

In [ ]:
class NANDGate(Neuron):
  def __init__(self):
    super().__init__([-3, 2, 2])

  def thresholdFunction(self, value):
    return value < 0

In [ ]:
gates = [
         ANDGate(),
         ORGate(),
         XORGate(),
         NORGate(),
         NANDGate()
]

In [ ]:
inputs = [
          [False, False],
          [False, True],
          [True, False],
          [True, True]
]

In [ ]:
for gate in gates:
  print(type(gate).__name__, "outputs:")
  for input in inputs:
    print("\t", input, "->", gate.process(input))
  print()

## Assignment 5
Boolean functions using Single Layer Perceptron

In [ ]:
import numpy as np

In [ ]:
class Perceptron:

  def __init__(self, input_size):
    self.input_size = input_size
    self.w = [np.random.rand() for _ in range(input_size)]
    self.b = np.random.rand()
  def __activation(self, y):
    return int(y >= 0)

  def __forward_prop(self, X):
    z = self.b
    for i in range(self.input_size):
      z += self.w[i] * X[i]
    return z

  def __backward_prop(self, X, y, y_hat, learning_rate):
    delta_w = learning_rate * (y - y_hat)
    for i in range(self.input_size):
      delta_wi = delta_w*X[i]
      self.w[i] += delta_wi
    delta_b = delta_w
    self.b += delta_b

  def __train(self, X, y, learning_rate):
    z = self.__forward_prop(X)
    y_hat = self.__activation(z)
    self.__backward_prop(X, y, y_hat, learning_rate)
    

  def fit(self, X, y, learning_rate, epochs = 5):
    for _ in range(epochs):
      for i in range(len(X)):
        self.__train(X[i], y[i], learning_rate)

  def predict(self, X):
    z = self.__forward_prop(X)
    return self.__activation(z)

In [ ]:
X = [
     [0, 0],
     [0, 1],
     [1, 0],
     [1, 1]
]

In [ ]:
bool_functions = [
      Perceptron(2), # OR
      Perceptron(2), # AND
      Perceptron(2), # NOR
      Perceptron(2), # NAND
]

In [ ]:
y = [
      [0, 1, 1, 1],
      [0, 0, 0, 1],
      [1, 0, 0, 0],
      [1, 1, 1, 0]
]

In [ ]:
for i in range(len(bool_functions)):
  bool_function = bool_functions[i]
  input = X[:]
  output = y[i][:]
  bool_function.fit(input, output, 0.01, epochs=100)
  for inputi in input:
    print(inputi, "->", bool_function.predict(inputi))
  print()

## Assignment 7
Single hidden layer neural network

In [ ]:
import numpy as np

In [ ]:
class NeuralNetwork:
  def __init__(self, input_nodes, hidden_nodes):
    self.layers = [input_nodes, hidden_nodes, 1]
    self.__init_weights()

  def __init_weights(self):
    self.W = np.array([None for _ in range(len(self.layers) - 1)])
    self.b = np.array([None for _ in range(len(self.layers) - 1)])
    for i in range(len(self.layers) - 1):
      self.W[i] = np.ones((self.layers[i+1], self.layers[i]))
      self.b[i] = np.zeros((self.layers[i+1]))

  def __train(self, X, y):
    a = np.array([None for _ in range(len(self.layers) - 1)])
    a[0] = X.copy()
    a[1] = np.dot(self.W[0], a[0]) + self.b[0]
    
    self.W[1] -= -a[1]
    self.b[1] -= -1
    for i in range(len(a[0])):
      self.W[0][:, i] -= -np.sum(self.W[1] * a[0][i], axis = 0)
    self.b[0] -= -1

  def fit(self, X, y):
    for i in range(len(X)):
      self.__train(X[i], y[i])

In [ ]:
model = NeuralNetwork(2, 2)

In [ ]:
model.fit([
           np.array([1, 0])
], [1])

In [ ]:
print(f'Value of W11: {model.W[0][0,0]}')

## Assignment 8
Particle Swarm Optimization on Travelling Salesman Problem

In [ ]:
import random

In [ ]:
def toss(p):
  return random.randint(0, 100) <= p

In [ ]:
class SwapOperator:
  def __init__(self, i, j):
    self.i = i
    self.j = j
  
  def get_max(self):
    return max(self.i, self.j)

  def __str__(self):
    pair = (self.i, self.j)
    return str(pair)

  def __repr__(self):
    pair = (self.i, self.j)
    return str(pair)

In [ ]:
class SwapSequence:
  def __init__(self, seq):
    self.seq = seq # List of SwapOperator objects

  def iterate(self):
    for so in self.seq:
      yield so
  
  def add_swap_operator(self, so):
    self.seq.append(so)

  def get_highest_index(self):
    hi = 0
    for so in self.seq:
      hi = max(hi, so.get_max())
    return hi

  def copy(self):
    ret = []
    for so in self.seq:
      ret.append(SwapOperator(so.i, so.j))
    return SwapSequence(ret)

  def __add__(self, b): # Returns *Basic SwapSequence* that is equivalent to list(Sequence a) + list(Sequence b)
    c_seq = self.seq + b.seq
    c = SwapSequence(c_seq)
    n = c.get_highest_index() + 1

    initial_solution = Solution.gen_random_Solution(n)
    resultant_solution = initial_solution.add(c)

    return resultant_solution.sub(initial_solution)

  def __str__(self):
    return str(self.seq)

In [ ]:
class Solution:
  def __init__(self, perm):
    self.perm = perm # A basic permutation

  def __add_SO(self, so, p):
    ret = self.copy()
    if toss(p):
      ret.perm[so.i], ret.perm[so.j] = ret.perm[so.j], ret.perm[so.i]
    return ret

  def __add_SS(self, ss, p):
    ret = self.copy()
    for so in ss.iterate():
      ret = ret.add(so, p)
    return ret

  def add(self, obj, p = 100):
    if isinstance(obj, SwapOperator):
      return self.__add_SO(obj, p)
    elif isinstance(obj, SwapSequence):
      return self.__add_SS(obj, p)
    else: return None
  
  def sub(self, b, p = 100):
    b = b.copy()
    a = self.copy()
    n = len(a.perm)
    ss = []
    for i in range(n):
      for j in range(n):
        if a.perm[i] == b.perm[j]:
          if i != j:
            so = SwapOperator(i, j)
            ss.append(so)
            b = b.add(so, p)
          else:
            break
    return SwapSequence(ss)

  def get(self, ind):
    return self.perm[ind]

  def copy(self):
    return Solution(self.perm[:])

  def __str__(self):
    return str(self.perm)

  def __eq__(self, b):
    for i in range(len(self.perm)):
      if self.get(i) != b.get(i):
        return False
    return True

  @classmethod
  def gen_random_Solution(cls, n):
    perm = [i for i in range(n)]
    random.shuffle(perm)
    return Solution(perm)

In [ ]:
class Particle:
  def __init__(self, position, velocity, tsp):
    self.position = position
    self.p_best = self.position.copy()
    self.velocity = velocity
    self.fitness = tsp.apply_solution(position)
    self.p_best_fitness = self.fitness

  def get_fitness(self):
    return self.fitness
  
  def get_position(self):
    return self.position

  def get_p_best(self):
    return self.p_best

  def get_p_best_fitness(self):
    return self.p_best_fitness
  
  def update_velocity(self, g_best, alpha, beta):
    ini = self.velocity.copy()
    term1 = self.p_best.sub(self.position, alpha)
    term2 = g_best.sub(self.position, beta)
    self.velocity += term1 + term2

  def update_position(self, tsp):
    initial = self.position
    new = self.position.add(self.velocity)

    n_fitness = tsp.apply_solution(new)

    if n_fitness < self.get_p_best_fitness():
      self.p_best = new.copy()
      self.p_best_fitness = n_fitness

    self.fitness = n_fitness
    self.position = new

  def __str__(self):
    ret = f"Particle: {self.position}, fitness: {self.get_fitness()}, best_fitness: {self.get_p_best_fitness()}"
    return ret
  
  def __repr__(self):
    ret = f"Particle: {self.position}, fitness: {self.get_fitness()}, best_fitness: {self.get_p_best_fitness()}"
    return ret

  @classmethod
  def gen_random_Particle(self, n, tsp):
    position = Solution.gen_random_Solution(n)
    velocity = Solution.gen_random_Solution(n).sub(Solution.gen_random_Solution(n))
    return Particle(position, velocity, tsp)

In [ ]:
class TSP:
  def __init__(self, g):
    self.g = g
    self.n = len(g)

  def apply_solution(self, s):
    cost = 0
    for i in range(self.n):
      cost += self.g[s.get(i)][s.get((i + 1) % self.n)]
    return cost
  
  @classmethod
  def gen_random_TSP(cls, n, soln):
    ret = []
    for i in range(n):
      row = []
      for j in range(n):
        row.append(random.randint(2, 10))
      ret.append(row)
      ret[i][i] = 0
    soln = soln.perm
    for i in range(n):
      ret[soln[i]][soln[(i+1)%n]] = 1
    return TSP(ret)

In [ ]:
def get_g_best(particles, g_best):
  g_best, g_best_fitness = g_best
  for particle in particles:
    if particle.get_p_best_fitness() < g_best_fitness:
      g_best = particle.get_p_best().copy()
      g_best_fitness = particle.get_p_best_fitness()
  return g_best, g_best_fitness

In [ ]:
ALPHA = 70
BETA = 80
POPULATION_SIZE = 20
NUM_NODES = 10
NUM_ITERATIONS = 1000

In [ ]:
actual_solution = [i for i in range(NUM_NODES)]
random.shuffle(actual_solution)
actual_solution = Solution(actual_solution)

In [ ]:
tsp = TSP.gen_random_TSP(NUM_NODES, actual_solution)
particles = [Particle.gen_random_Particle(NUM_NODES, tsp) for _ in range(POPULATION_SIZE)]
g_best_fitness = particles[0].get_fitness()
g_best, g_best_fitness = get_g_best(particles, (particles[0], g_best_fitness))

In [ ]:
print("Initial Particles: ")
for particle in particles:
  print(particle)
print(f"Best Solution: {g_best}, Fitness: {g_best_fitness}")

In [ ]:
for i in range(NUM_ITERATIONS):
  for particle in particles:
    particle.update_velocity(g_best, ALPHA, BETA)
    particle.update_position(tsp)
    g_best, g_best_fitness = get_g_best(particles, (g_best, g_best_fitness))
  print(f"Iteration {i}, Best Solution: {g_best}, Fitness: {g_best_fitness}")

In [ ]:
print(actual_solution)
print(g_best)